In [ ]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [ ]:
get_ipython().system('pip install db-dtypes')
get_ipython().system('pip install xgboost')
get_ipython().system('pip install impyute')
get_ipython().system('pip install prophet')
get_ipython().system('pip install plotly')
get_ipython().system('pip install plotly-geo')

In [ ]:
import datetime
import itertools
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.figure_factory

In [ ]:
import statsmodels.api as sm
import sklearn.experimental.enable_iterative_imputer
import sklearn.impute
import sklearn.ensemble
import sklearn.model_selection
import sklearn.linear_model
import xgboost as xgb

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)

In [ ]:
# load data
def query(table):
    bigquery_client = bigquery.Client(project='ironhacks-data')
    query_str = f'''
SELECT *
FROM `ironhacks-data.ironhacks_competition.{table}`
'''
    query_job = bigquery_client.query(query_str)
    data = query_job.to_dataframe()
    return data

In [ ]:
def combine(u, w):
    '''
    Joins the unemployment data and the wage data on `uu_id`
    '''
    ww = w.loc[:, ['uu_id', 'average_wage']]
    d = u.join(ww.set_index('uu_id'), on='uu_id')
    return d

In [ ]:
def load_raw(csv_name='0_raw.csv'):
    '''
    Loads the unemployment and wage data and does some basic cleaning
    '''
    if not os.path.isfile(csv_name):
        u = query('unemployment_data')
        w = query('wage_data')
        raw = combine(u, w)
        raw.to_csv(csv_name, index=False)
    else:
        raw = pd.read_csv(csv_name)
    raw = raw.drop(['tract', 'timeperiod'], axis=1)
    raw = raw.sort_values(by=['uu_id', 'week_number'])
    raw = raw.drop_duplicates()
    raw = raw.replace({np.nan: None})
    raw = raw.reset_index(0, drop=True)
    return raw

In [ ]:
load_raw().to_dict('records')[0]

In [ ]:
# define relevant columns based on categories 
COL_MAP = {
    'edu': ['edu_8th_or_less', 'edu_grades_9_11', 'edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown'],
    'race': ['race_amerindian', 'race_asian', 'race_black', 'race_noanswer', 'race_hawaiiannative', 'race_other', 'race_white'],    
    'gender': ['gender_female', 'gender_male', 'gender_na'],
    'industry': ['top_category_employer1', 'top_category_employer2', 'top_category_employer3']        
}

In [ ]:
def get_cols(names):
    l = []
    for name in names:
        if name in COL_MAP:
            l += COL_MAP[name]
        else:
            l += [name]
    return l

In [ ]:
def subset(df, uu_id):        
    return df.loc[df.uu_id == df.uu_id.unique()[uu_id], :]

In [ ]:
def convert_to_submission(results_csv, week_number_to_submit=40):
    r = pd.read_csv(results_csv)
    last = r.loc[r.week_number == week_number_to_submit, ['uu_id', 'predicted']]
    last.index = last.uu_id
    uuid_map = last.to_dict(orient='dict')['predicted']
    p = query('prediction_list')
    p['total_claims'] = p['uu_id'].map(uuid_map)
    p.to_csv('submission_prediction_output.csv', index=False)

In [ ]:
def get_week_number_map(g, colname):
    '''
    Creates a dictionary that maps from week number to an existing value in a given `colname`
    '''
    g = g[['week_number', colname]]
    week_number_map = dict(sorted(g.values.tolist()))
    return week_number_map

In [ ]:
def get_county(tract_name):
    m = re.search('Census Tract \S+, (.+) County, Indiana', tract_name)
    county = m.group(1)
    return county

In [ ]:
def week_number_to_date(week_number, first_week_date='20220101'):
    '''
    Prepare a date column for ARIMA
    '''
    return pd.to_datetime(first_week_date, format='%Y%m%d') + pd.DateOffset(days=7*(week_number - 1))

In [ ]:
def insert_na_week_number(g, max_week_number=37):    
    d = {}
    for colname in g.columns:
        if colname == 'week_number':
            continue
        week_number_map = get_week_number_map(g, colname)
        series = pd.Series(range(1, max_week_number+1))        
        d[colname] = series.map(week_number_map)        
        
    df = pd.DataFrame(d)
    df['week_number'] = range(1, max_week_number+1)
    df['uu_id'] = [v for v in df['uu_id'].unique() if type(v) == str][0]
    df['average_wage'] = [v for v in g['average_wage'].unique()][0]
    df['countyfips'] = [v for v in g['countyfips'].unique()][0]
    df['county'] = get_county([v for v in g['tract_name'].unique()][0])
    return df

In [ ]:
def load_raw_full(csv_name='1_raw_full.csv'):
    if not os.path.isfile(csv_name):
        raw = load_raw()
        raw_full = raw.groupby('uu_id').apply(insert_na_week_number).reset_index(0, drop=True)
        raw_full['date'] = raw_full['week_number'].apply(week_number_to_date)
        raw_full = raw_full.drop(['tract_name'], axis=1)
        raw_full.to_csv(csv_name, index=False)        
    else:
        raw_full = pd.read_csv(csv_name)
    return raw_full

In [ ]:
load_raw_full().to_dict('records')[0]

In [ ]:
def plot_impute(n=6):
    raw = load_raw_full()
    imp = load_imp_tot()
    fig, axs = plt.subplots(ncols=n, sharey=False, figsize=(n*3, 3))
    for i in range(n):
        rraw = subset(raw, i)
        iimp = subset(imp, i)
        ax = axs[i]
        ax.plot(rraw.week_number, rraw.total_claims, 'o', label='original')
        ax.plot(iimp.week_number, iimp.total_claims, '-', label='imputed')
        ax.set_xlim(0, 42)
    axs[-1].legend(fancybox=False)

In [ ]:
def imp_tot(g):
    g['total_claims'] = g['total_claims'].interpolate(method='linear')
    g = g[g['total_claims'].notna()]
    return g

In [ ]:
def load_imp_tot(csv_name='2_imp_tot.csv'):
    if not os.path.isfile(csv_name):
        d = load_raw_full()
        d = d.groupby(['uu_id']).apply(imp_tot).reset_index(0, drop=True)
        d.to_csv(csv_name, index=False)
    else:
        d = pd.read_csv(csv_name)
    return d

In [ ]:
plot_impute()

In [ ]:
def replace_na_cols(g):
    '''
    If a column only has None or zero values, replace that entire columnn with zeros
    '''
    x = g.copy()
    for col in g.columns:
        cond1 = g[col].isnull()
        cond2 = g[col] == 0
        if (cond1 | cond2).all():
            x[col] = 0        
    return x

In [ ]:
def impute_iterative(df):
    '''
    Wrapper fucntion for IterativeImputer for a generic data frame. 
    Mostly, for testing. We might need need this function
    Impute data assuming there are zero columns where all the values are NA
    '''
    imputer = sklearn.impute.IterativeImputer(random_state=0, min_value=0)
    imputed_cols = imputer.fit_transform(df)
    df_imputed = pd.DataFrame(imputed_cols, columns=df.columns)
    return df_imputed

In [ ]:
def iter_cat(g):
    g = replace_na_cols(g)    
    for cat in ['edu', 'race', 'gender']:
        gg = g.loc[:, COL_MAP[cat] + ['total_claims']]
        yield cat, gg

In [ ]:
def print_impute_cat(test_subset, impute_func):
    for cat, gg in iter_cat(test_subset):
        line = '*'*len(cat)
        print(line)
        print(cat)
        print(line)
        print(impute_func(gg).head())

In [ ]:
print_impute_cat(subset(load_imp_tot(), 6), impute_iterative)

In [ ]:
def impute_rowsum(df, target_col='total_claims'):
    l = []
    for idx, row in df.iterrows():
        n_unknowns = row.isna().sum()        
        if n_unknowns == 1:
            others = row[~row.isna() & (row.index != target_col)]
            val = row[target_col] - others.sum()
            val = val if val > 0 else 0
            row[row.isna()] = val
        l.append(row)
    df = pd.DataFrame(l).reset_index(0, drop=True)
    
    l = []
    for idx, row in df.iterrows():
        n_unknowns = row.isna().sum()        
        
        weights = {}
        for col in row[row.isna()].index:
            weights[col] = df[col].mean()
        weights = {k:v/sum(weights.values()) for k, v in weights.items()}       
        
        if n_unknowns > 1:            
            others = row[~row.isna() & (row.index != target_col)]
            row[row.isna()] = row[row.isna()].index.map(weights)*(row[target_col] - others.sum())            
        l.append(row)
        
    df_imputed = pd.DataFrame(l).reset_index(0, drop=True)
    return df_imputed

In [ ]:
print_impute_cat(subset(load_imp_tot(), 6), impute_rowsum)

In [ ]:
def impute_all(df):
    x = df.copy().reset_index(0, drop=True)
    for cat, gg in iter_cat(df):
        df_imputed = impute_rowsum(gg)
        df_imputed = df_imputed.drop('total_claims', axis=1)
        x[COL_MAP[cat]] = df_imputed
    return x

In [ ]:
def load_clean(csv_name='3_clean.csv'):    
    if not os.path.isfile(csv_name):
        imp_tot = load_imp_tot()
        imp = imp_tot.groupby('uu_id').apply(impute_all).reset_index(0, drop=True)
        imp['date'] = imp['week_number'].apply(week_number_to_date)
        imp.to_csv(csv_name, index=False)
    else:
        imp = pd.read_csv(csv_name)
    
    return imp

In [ ]:
load_clean().to_dict('records')[0]

In [ ]:
def impute_logistic(g, ycol):
    g = g[[ycol, 'week_number', 'total_claims']]
    xcols = ['week_number', 'total_claims']
    ycols = [ycol]
    
    mask_train = ~g[ycol].isnull()
    x_train, x_test = g.loc[mask_train, xcols], g.loc[~mask_train, xcols]
    y_train, y_test = g.loc[mask_train, ycols], g.loc[~mask_train, ycols]
    
    if y_train.shape[0] == 0:
        g[ycol] = None
        return g[ycol]
    
    classes = y_train[ycol].unique()
    if len(classes) == 1:
        yhat = [classes[0]]
    else:
        model = sklearn.linear_model.LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=10000).fit(x_train, y_train.values.ravel())
        yhat = model.predict(x_test)
    g.loc[~mask_train, ycols] = yhat
    return g[ycol]

In [ ]:
def impute_industry(g, max_week_number=37):
    g = g.loc[g.week_number <= max_week_number, :]
    x = g.copy()
    for colname in COL_MAP['industry']:
        x[colname] = impute_logistic(g, colname)
    return x

In [ ]:
def load_imp_industry(csv_name='4_imp_industry.csv'):
    if not os.path.isfile(csv_name):
        d = load_clean()
        d = d.groupby('uu_id').apply(impute_industry).reset_index(0, drop=True)
        d.to_csv(csv_name, index=False)
    else:
        d = pd.read_csv(csv_name)
    return d

In [ ]:
load_imp_industry().isna().sum()

In [ ]:
def feature_engineer(d):
    d['gender_male_ratio'] = d['gender_male'] / d['total_claims']
    d['edu_post_hs_ratio'] = d['edu_post_hs'] / d['total_claims']
    d['race_white_ratio'] = d['race_white'] / d['total_claims']
    d['race_black_ratio'] = d['race_black'] / d['total_claims']
    d = d.drop(get_cols(['gender', 'edu', 'race']), axis=1)
    return d

In [ ]:
def load_featured(csv_name='5_featured.csv'):
    if not os.path.isfile(csv_name):
        d = load_imp_industry()
        d = feature_engineer(d)
    else:
        d = pd.read_csv(csv_name)
    return d

In [ ]:
load_featured().to_dict('records')[0]

In [ ]:
def get_tot_by_county(d):
    l = []
    for (county, week_number), g in d.groupby(['county', 'week_number']):        
        l.append({
            'county': county,
            'fips': g.countyfips.values[0],
            'week_number': week_number,
            'total_claims': g.total_claims.mean()
        })
    c = pd.DataFrame(l)
    return c

In [ ]:
c = get_tot_by_county(load_featured())
c = c.loc[c.week_number == 20, :]
c.head()

In [ ]:
values = c.total_claims
endpts = list(np.mgrid[min(values):max(values):4j])
colorscale = ["#030512","#1d1d3b","#323268","#3d4b94","#3e6ab0",
              "#4989bc","#60a7c7","#85c5d3","#b7e0e4","#eafcfd"]
# fig = plotly.figure_factory.create_choropleth(
#     fips=c.fips, values=values, scope=['Indiana'], show_state_data=True,
#     colorscale=colorscale, binning_endpoints=endpts, round_legend_values=True,
#     plot_bgcolor='rgb(229,229,229)',
#     paper_bgcolor='rgb(229,229,229)',
#     legend_title='Average Total Claims by County',
#     county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
#     exponent_format=True,
# )

In [ ]:
values = c.total_claims
endpts = list(np.mgrid[min(values):max(values):4j])
colorscale = ["#030512","#1d1d3b","#323268","#3d4b94","#3e6ab0",
              "#4989bc","#60a7c7","#85c5d3","#b7e0e4","#eafcfd"]
fig = plotly.figure_factory.create_choropleth(
    fips=c.fips, values=values, scope=['Indiana'], show_state_data=True,
    colorscale=colorscale, binning_endpoints=endpts, round_legend_values=True,
    plot_bgcolor='rgb(229,229,229)',
    paper_bgcolor='rgb(229,229,229)',
    legend_title='Average Total Claims by County',
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
    exponent_format=True,
)

In [ ]:
get_ipython().system('pip install db-dtypes')
get_ipython().system('pip install xgboost')
get_ipython().system('pip install impyute')
get_ipython().system('pip install prophet')
get_ipython().system('pip install plotly')
get_ipython().system('pip install plotly-geo')
get_ipython().system('pip install geopandas')
get_ipython().system('pip install pyshp')
get_ipython().system('pip install shapely')